##### In this notebook
I pull out some test data. I make sure to include one of the 119 situation E cases (multiple offenders and multiple victims), so as to have partial lines which contain both offenders and victims. I manually create three dictionaries, to confirm that this is the format we are aiming for with my code. I can also use these to test my code against.

In [1]:
import re
import sys

In [2]:
with open('../data/2016 SHR FIle LIst.txt') as f:
    lines = f.readlines()

In [3]:
test = lines[2181-1:2191-1]  #Atom: three lines 2181:2190

In [4]:
test

['B                                                           PAGE  53\n',
 '1                                SUPPLEMENTARY HOMICIDE REPORT - FILE LISTING 2016                             UCR44300    07/07/17\n',
 ' SWITCH = N                                     BY STATE WITHIN GROUP\n',
 '0                                      VICTIM                 OFFENDER\n',
 '      AGENCY   G MO HOM  INC#  SIT    A  S  R  E    A  S  R  E  WEAP  REL  CIR SUB        AGENCY NAME         STATE\n',
 '0   04 CA03604 2 12  A    001   E    22  M  W  H   21  M  W  H   11    AQ   18       FONTANA                   CALIF\n',
 '                                     22  M  W  H   19  M  B  N   11    AQ   18\n',
 '0   04 CA03607 2 01  A    001   D    60  M  W  N   71  M  W  N   11    IL   26       ONTARIO                   CALIF\n',
 '                                     59  F  W  N\n',
 '    04 CA03607 2     A    002   A    33  M  W  H   37  M  W  H   11    FR   99       ONTARIO                   CALIF\n']

In [6]:
crime1a = test[5]
crime1b = test[6]
crime2a = test[7]
crime2b = test[8]
crime3 = test[9]

In [15]:
H = ["STATE_NUM","AGENCY","G","MO","HOM","INC_NUM","SIT", "VICTIMS", "OFFENDERS","AGENCY NAME","STATE"
]  #Added State_num field 

In [16]:
V = ["VA","VS","VR","VE"]

In [17]:
O = ["OA","OS","OR","OE","OWEAP","OREL","OCIR","OSUB"]

In [7]:
crime1a

'0   04 CA03604 2 12  A    001   E    22  M  W  H   21  M  W  H   11    AQ   18       FONTANA                   CALIF\n'

In [8]:
crime1b  #partial line with victim AND offender because 'SIT' == 'E'

'                                     22  M  W  H   19  M  B  N   11    AQ   18\n'

In [24]:
#length 2 list of dictionaries
crime1_v = [{ V[0]: '22', V[1]: 'M', V[2]: 'W', V[3]: 'H' }, { V[0]: '22', V[1]: 'M', V[2]: 'W', V[3]: 'H' }]
#length 2 list of dictionaries
crime1_o = [{ O[0]: '21', O[1]: 'M', O[2]: 'W', O[3]: 'H', O[4]: '11', O[5]: 'AQ', O[6]: '18', O[7]: ''}, { O[0]: '19', O[1]: 'M', O[2]: 'B', O[3]: 'N', O[4]: '11', O[5]: 'AQ', O[6]: '18', O[7]: ''}] 
crime1_dict = { H[0]: '04', H[1]: 'CA03604', H[2]: '2', H[3]: '12', H[4]: 'A', H[5]: '001', H[6]: 'E', H[7]: crime1_v, H[8]: crime1_o, H[9]: 'FONTANA', H[10]: 'CALIF'}

In [25]:
crime1_dict

{'AGENCY': 'CA03604',
 'AGENCY NAME': 'FONTANA',
 'G': '2',
 'HOM': 'A',
 'INC_NUM': '001',
 'MO': '12',
 'OFFENDERS': [{'OA': '21',
   'OCIR': '18',
   'OE': 'H',
   'OR': 'W',
   'OREL': 'AQ',
   'OS': 'M',
   'OSUB': '',
   'OWEAP': '11'},
  {'OA': '19',
   'OCIR': '18',
   'OE': 'N',
   'OR': 'B',
   'OREL': 'AQ',
   'OS': 'M',
   'OSUB': '',
   'OWEAP': '11'}],
 'SIT': 'E',
 'STATE': 'CALIF',
 'STATE_NUM': '04',
 'VICTIMS': [{'VA': '22', 'VE': 'H', 'VR': 'W', 'VS': 'M'},
  {'VA': '22', 'VE': 'H', 'VR': 'W', 'VS': 'M'}]}

In [11]:
crime2a 

'0   04 CA03607 2 01  A    001   D    60  M  W  N   71  M  W  N   11    IL   26       ONTARIO                   CALIF\n'

In [12]:
crime2b

'                                     59  F  W  N\n'

In [20]:
#length 2 list of victim dictionaries               
crime2_v = [{ V[0]: '60', V[1]: 'M', V[2]: 'W', V[3]: 'N' }, {V[0]: '59', V[1]: 'F', V[2]: 'W', V[3]: 'N'}]
#length 1 list of offender dictionaries
crime2_o = [{ O[0]: '71', O[1]: 'M', O[2]: 'W', O[3]: 'N', O[4]: '11', O[5]: 'IL', O[6]: '26', O[7]: '', }] 
crime2_dict = { H[0]: '04', H[1]: 'CA03607', H[2]: '2', H[3]: '01', H[4]: 'A', H[5]: '001', H[6]: 'D', H[7]: crime2_v, H[8]: crime2_o, H[9]: 'ONTARIO', H[10]: 'CALIF'}  

In [23]:
crime2_dict

{'AGENCY': 'CA03607',
 'AGENCY NAME': 'ONTARIO',
 'G': '2',
 'HOM': 'A',
 'INC_NUM': '001',
 'MO': '01',
 'OFFENDERS': [{'OA': '71',
   'OCIR': '26',
   'OE': 'N',
   'OR': 'W',
   'OREL': 'IL',
   'OS': 'M',
   'OSUB': '',
   'OWEAP': '11'}],
 'SIT': 'D',
 'STATE': 'CALIF',
 'STATE_NUM': '04',
 'VICTIMS': [{'VA': '60', 'VE': 'N', 'VR': 'W', 'VS': 'M'},
  {'VA': '59', 'VE': 'N', 'VR': 'W', 'VS': 'F'}]}

##### Interpreting the dictionary. Just for fun: crime 2 in the style of hard-boiled detective fiction
2016's been a quiet year so far for the cops in Ontario, CA. Not too much crime has spilled east from LA. How long can it last? How long you think? Mid-January: gruesome double homicide.
Some old white dude went round to visit his younger sister. She wasn't in, but her husband of thirty years was and he had company. It looks like he probably didn't like that.

In [13]:
crime3

'    04 CA03607 2     A    002   A    33  M  W  H   37  M  W  H   11    FR   99       ONTARIO                   CALIF\n'

In [26]:
#length 1 list of offender dictionaries
crime3_v = [{ V[0]: '33', V[1]: 'M', V[2]: 'W', V[3]: 'H' }]
#length 1 list of offender dictionaries
crime3_o = [{ O[0]: '37', O[1]: 'M', O[2]: 'W', O[3]: 'H', O[4]: '11', O[5]: 'FR', O[6]: '99', O[7]: '', }] 
#When month is empty, we want to bring it down from previous crime
crime3_dict = { H[0]: '04', H[1]: 'CA03607', H[2]: '2', H[3]: '01', H[4]: 'A', H[5]: '002', H[6]: 'A', H[7]: crime3_v, H[8]: crime3_o, H[9]: 'ONTARIO', H[10]: 'CALIF'}  

In [27]:
crime3_dict

{'AGENCY': 'CA03607',
 'AGENCY NAME': 'ONTARIO',
 'G': '2',
 'HOM': 'A',
 'INC_NUM': '002',
 'MO': '01',
 'OFFENDERS': [{'OA': '37',
   'OCIR': '99',
   'OE': 'H',
   'OR': 'W',
   'OREL': 'FR',
   'OS': 'M',
   'OSUB': '',
   'OWEAP': '11'}],
 'SIT': 'A',
 'STATE': 'CALIF',
 'STATE_NUM': '04',
 'VICTIMS': [{'VA': '33', 'VE': 'H', 'VR': 'W', 'VS': 'M'}]}

Question 1: Do we want all dictionary values to be strings, even if they are actually integers - e.g. VA.

Question 2: Do we have an idea what the random 0 at the start of the line denotes? And how to deal with it when looping over lines?

Question 3: Am I correct to code empty cells as '' ?

Question 4: Ultimately, what should I do with individual crime dictionaries? Put them each in separate files? Or collect them all into one big object (such as dictionary)?